<a href="https://colab.research.google.com/github/JimMiller-0/Might-be-my-Year/blob/main/Might_be_my_Year.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a notebook for retreiving Fantasy Football Data from ESPN for analysis and predictive capabilities

Get Data

In [ ]:
# Install necessary SDKs

!pip install espn-api
!pip install google-cloud-secret-manager
!pip install google-cloud-bigquery





Imports

In [34]:
import pandas as pd
from espn_api.football import League
from google.cloud import secretmanager
import requests
import json
import time
import numpy as np
import datetime
import os

League Variables & Access Tokens for ESPN API

In [35]:
# You will need to get your league ID and ESPN S2 and SWID
# See https://github.com/cwendt94/espn-api/wiki/Football-Intro for details
# Recommended: Store SWID and ESPN S2 in a secrets manager, like gcp secrets manager: https://cloud.google.com/security/products/secret-manager

league_id = 1054374 # => set to league ID that you want to pull data from
season=2023 # => set to year you want to pull data from
url=f'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{season}/segments/0/leagues/{league_id}?scoringPeriodId=17&view=mBoxscore&view=m' # => url of ESPN API. Note: this has changed over the years, navigate to the site and inspect network calls to get current endpoint

# Authenticate to Google Cloud
from google.colab import auth
auth.authenticate_user()

#Create a Client for secrets manager
client = secretmanager.SecretManagerServiceClient()
project_id = 'might-be-my-year' # => GCP Project ID where secrets manager is enabled
secret_espn_s2 = 'espn_s2' # name of secret in GCP secrets manager for espn_s2
secret_swid = 'swid' # name of secret in GCP secrets manager for swid

# Forge the paths to the latest version of the secrets with a F-string:
resource_name_espn_s2 = f"projects/{project_id}/secrets/{secret_espn_s2}/versions/latest"
resource_name_swid = f"projects/{project_id}/secrets/{secret_swid}/versions/latest"

# Load up the secrets to a variable at runtime:
response_espn_s2 = client.access_secret_version(name=resource_name_espn_s2)
response_swid = client.access_secret_version(name=resource_name_swid)

espn_s2 = response_espn_s2.payload.data.decode("UTF-8")
swid = response_swid.payload.data.decode("UTF-8")





Get Data From ESPN API

In [ ]:
r = requests.get(url, cookies={'swid': swid, 'espn_s2': espn_s2})
if r.status_code == 200:

  data = r.json()
  data

# Save the data to a JSON file
  with open('data.json', 'w') as f:
        json.dump(data, f, indent=0)  # Use indent for pretty printing
  print('JSON file saved successfully.')
else:
  print('Request failed with status code:', r.status_code)

# Read the JSON file
with open('data.json', 'r') as f:
    data = json.load(f)

data


Get Data in a schema that makes sense because ESPN is crazy.

In [ ]:
# test to make sure pandas can put the json into a dataframe
df = pd.json_normalize(data, record_path=['teams'])
df

In [ ]:
# Get Draft Details

draft_url= f'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{season}/segments/0/leagues/{league_id}?view=mDraftDetail&view=mSettings&view=mTeam&view=modular&view=mNav'

r = requests.get(draft_url, cookies={'swid': swid, 'espn_s2': espn_s2})
if r.status_code == 200:

  draft_data = r.json()
  draft_data

# Save the data to a JSON file
  with open('draft_data.json', 'w') as f:
        json.dump(draft_data, f, indent=0)  # Use indent for pretty printing
  print('JSON file saved successfully.')
else:
  print('Request failed with status code:', r.status_code)

  # Read the JSON file
with open('draft_data.json', 'r') as f:
    draft_data = json.load(f)

draft_picks_df = pd.json_normalize(draft_data['draftDetail'], record_path=['picks'])
draft_members_df = pd.json_normalize(draft_data, record_path=['members'])
draft_settings_df = pd.json_normalize(draft_data['settings'])
draft_status_df = pd.json_normalize(draft_data['status'])
draft_teams_df = pd.json_normalize(draft_data, record_path=['teams'])

# test to make sure dataframes are working
draft_teams_df


Load Data in BigQuery - This makes is easier to manually slice and dice

In [70]:
from google.cloud import bigquery
import pyarrow as pa
from google.cloud.exceptions import NotFound
import pyarrow.parquet as pq

# set project id again
project_id = 'might-be-my-year'  # Replace with your actual project ID

# Construct a BigQuery client object.
bq_client = bigquery.Client(project=project_id)

# TODO(developer): Set dataset_id to the ID of the dataset to create.
dataset_id = "{}.fantasy_football".format(bq_client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# TODO(developer): Specify the geographic location where the dataset should reside.
dataset.location = "US"

# Check if the dataset exists
try:
    bq_client.get_dataset(dataset_id)  # Make an API request.
    print(f"Dataset {dataset_id} already exists.")
except NotFound:
    print(f"Dataset {dataset_id} does not exist. Creating...")
# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
    dataset = bq_client.create_dataset(dataset, timeout=30)  # Make an API request.
    print("Created dataset {}.{}".format(bq_client.project, dataset.dataset_id))

# create lists of all dataframes with coooresponding table names
all_dfs = [draft_picks_df, draft_members_df, draft_settings_df, draft_status_df, draft_teams_df]
table_names = ['draft_picks', 'draft_members', 'draft_settings', 'draft_status', 'draft_teams']

for df, table_name in zip(all_dfs, table_names):

# Replace periods in column names with underscores
  df.columns = df.columns.str.replace('.', '_')  # Replace '.' with '_' in column names because thats what BQ needs
# Convert the DataFrame to a Parquet file
  table = pa.Table.from_pandas(df)
  pq.write_table(table, f'{table_name}.parquet')

# TODO(developer): Set table_id to the ID of the table to create.
  table_id = f'{dataset_id}.{table_name}_{season}'

  job_config = bigquery.LoadJobConfig(
      autodetect=True, source_format=bigquery.SourceFormat.PARQUET
  )

# open parquet file -> load the table from parquet file into bq table
  with open(f'{table_name}.parquet', "rb") as source_file:
      job = bq_client.load_table_from_file(
          source_file, table_id, job_config=job_config
      )

# Wait for the job to complete
  job.result()

  print("Loaded {} rows and {} columns to {}".format(job.output_rows, len(df.columns), table_id))

Dataset might-be-my-year.fantasy_football already exists.
Loaded 180 rows and 14 columns to might-be-my-year.fantasy_football.draft_picks_2023
Loaded 12 rows and 7 columns to might-be-my-year.fantasy_football.draft_members_2023
Loaded 1 rows and 131 columns to might-be-my-year.fantasy_football.draft_settings_2023
Loaded 1 rows and 56 columns to might-be-my-year.fantasy_football.draft_status_2023
Loaded 12 rows and 125 columns to might-be-my-year.fantasy_football.draft_teams_2023


Analyze Data

Goal: Plot "Winners" and "Losers" based off of draft day auction price vs. production.

should be somthing like avg auction value vs price paid for on draft day vs inferred auction value based on end of year performance

plot 1: avg auction value vs price paid for on draft day. + difference shows league values that player/position more, - difference league values that player/position less

plot 2: avg auction value vs inferred auction value based on end of the year performance. + difference = player outperfomed expectations, - difference = player underperformed expectations

Plot 3: differnece in plot 1 vs difference in plot 2. quadrant plot: q1 =  players who under performed and the league overvalued. q2 = players who out performed and the league over valued. q3 = players who outperformed and the league undervalued. q4 =  players that underperformed and the league undervalued


In [ ]:
#